In [115]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [116]:
# Tree Node

class Node:
    # Represents a node in the decision tree
    # feature - which attribute to test
    # branches - dictionary mapping feature values to child nodes
    # value - the prediction (only for leaf nodes)
    def __init__(self, feature=None, branches=None, value=None):
        self.feature = feature  # The feature to split on at this node
        self.branches = branches or {}  # Subtrees for each feature value
        self.value = value  # Prediction stored at leaf nodes

In [117]:
# DecisionTreeID3

class DecisionTreeID3:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth  # Maximum depth of the tree, limits complexity
        self.root = None  # Root node of the tree


    def entropy(self, labels):
        # Calculate entropy of the target label
        probs = labels.value_counts(normalize=True)  # Proportion of each class in label
        return -sum(probs * np.log2(probs))  # Entropy formula: -Σp*log2(p)


    def information_gain(self, labels, labels_split):
        # Calculate information gain from a split
        parent_entropy = self.entropy(labels)  # Entropy before splitting
        # Weighted sum of entropies of the child nodes
        weighted_entropy = sum((len(subset) / len(labels)) * self.entropy(subset) for subset in labels_split)
        return parent_entropy - weighted_entropy  # Reduction in entropy after the split


    def find_best_feature(self, features, labels):
        # Find the feature that provides the best split
        best_feature = None  # The feature with the highest information gain
        best_gain = -1  # Track the highest information gain
        best_splits = None  # Store the resulting splits for the best feature

        for feature in features.columns:  # Iterate through each feature
            # Create splits by grouping the data by unique feature values
            splits = {value: labels[features[feature] == value] for value in features[feature].unique()}
            labels_split = list(splits.values())  # Convert to list of Series for calculation

            gain = self.information_gain(labels, labels_split)  # Calculate information gain for the feature
            if gain > best_gain:  # Update if the current feature has better information gain
                best_gain = gain
                best_feature = feature
                best_splits = splits

        return best_feature, best_splits  # Return the feature and its splits


    def build_tree(self, features, labels, depth=0):
        # Recursively build the decision tree
        if len(set(labels)) == 1:  # If all target labels are the same, it's a pure node
            return Node(value=labels.iloc[0])  # Create a leaf node with the target value
        if not features.columns.size or (self.max_depth and depth >= self.max_depth):
            # If no features left or max depth reached, return majority class
            return Node(value=labels.value_counts().idxmax())  # Most common target label

        # Find the best feature and splits for this level
        best_feature, best_splits = self.find_best_feature(features, labels)
        if not best_feature:  # If no meaningful split is found, return majority class
            return Node(value=labels.value_counts().idxmax())

        # Create a decision node with branches for each unique value of the best feature
        branches = {}
        for value, subset in best_splits.items():  # For each feature value
            subset_features = features[features[best_feature] == value].drop(columns=[best_feature])  # Remove used feature
            branches[value] = self.build_tree(subset_features, subset, depth + 1)  # Recursively build the tree

        return Node(feature=best_feature, branches=branches)  # Return the decision node



    def fit(self, features, labels):
        # Train the decision tree by building it from the data
        self.root = self.build_tree(features, labels)  # Build the tree starting from the root


    def predict_single(self, node, feature):
        # Predict the label for a single instance by traversing the tree
        if node.value is not None:  # If it's a leaf node, return the stored value
            return node.value
        feature_value = feature[node.feature]  # Get the value of the feature at this node
        if feature_value in node.branches:  # Check if there's a branch for this value
            return self.predict_single(node.branches[feature_value], feature)  # Traverse to the child node
        return None  # If no branch matches, return None (optional: handle this case separately)


    def predict(self, features):
        # Predict the labels for multiple instances
        return [self.predict_single(self.root, feature) for _, feature in features.iterrows()]  # Apply predict_single to each row

In [119]:
# Visulalise Decision Tree 

def visualise_DT(tree):
    """
    Creates a horizontal tree visualization using NetworkX
    """
    G = nx.Graph()
    pos = {}
    labels = {}
    
    def add_nodes(node, x=0, y=0, parent=None, level=0):
        node_id = str(id(node))
        
        # Create node label
        if node.value is not None:
            label = f"Predict: {node.value}"
        else:
            label = f"Split: {node.feature}"
            
        # Add node
        G.add_node(node_id)
        pos[node_id] = (x, y)
        labels[node_id] = label
        
        # Add edge from parent
        if parent:
            G.add_edge(parent, node_id)
        
        # Process children with vertical spacing
        if node.branches:
            children = list(node.branches.items())
            total_height = len(children)
            for i, (value, child) in enumerate(children):
                # Calculate position for child
                child_y = y - (total_height/2) + i
                child_x = x + 2  # Fixed horizontal spacing
                
                # Add child node
                child_id = add_nodes(child, child_x, child_y, node_id, level+1)
                
                # Add edge label
                edge_center = ((x + child_x)/2, (y + child_y)/2)
                plt.text(edge_center[0], edge_center[1], value, 
                        ha='center', va='center',
                        bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
                
        return node_id
    
    plt.figure(figsize=(15, 10))
    
    # Build graph starting from root
    add_nodes(tree.root)
    
    # Draw the graph
    nx.draw(G, pos,
            labels=labels,
            with_labels=True,
            node_color='lightblue',
            node_size=3000,
            font_size=8,
            font_weight='bold',
            font_color='black',
            width=1,
            edge_color='gray')
    
    plt.axis('off')
    plt.title("Decision Tree (Horizontal Layout)")
    plt.tight_layout()
    plt.show()

In [ ]:
# Train, Test and Visualise - Weather Data

data = pd.read_csv('weather-data.csv')

features = data.drop(columns=['Decision', 'Day'])  # Remove both Decision and Day
labels = data['Decision']

tree = DecisionTreeID3(max_depth=3)
tree.fit(features, labels)

predictions = tree.predict(features)
print("Predictions:", predictions)

accuracy = sum(pred == actual for pred, actual in zip(predictions, labels)) / len(labels)
print(f"Accuracy: {accuracy:.2%}")

visualise_DT(tree)

In [ ]:
# Train, Test and Visualise - Mushroom Data

data = pd.read_csv('mushrooms.csv')

features = data.drop(columns=['class']) 
labels = data['class']

tree = DecisionTreeID3(max_depth=3)
tree.fit(features, labels)

predictions = tree.predict(features)
print("Predictions:", predictions)

accuracy = sum(pred == actual for pred, actual in zip(predictions, labels)) / len(labels)
print(f"Accuracy: {accuracy:.2%}")

visualise_DT(tree)